In [1]:
%%writefile topsis.py
import pandas as pd
import numpy as np
import sys
import os

def main():
    if len(sys.argv) != 5:
        print("Error: Invalid number of arguments.")
        print("Usage: python topsis.py <InputDataFile> <Weights> <Impacts> <OutputResultFileName>")
        return

    input_file = sys.argv[1]
    weights_str = sys.argv[2]
    impacts_str = sys.argv[3]
    output_file = sys.argv[4]

    if not os.path.isfile(input_file):
        print(f"Error: File '{input_file}' not found.")
        return

    try:
        df = pd.read_csv(input_file)

        if len(df.columns) < 3:
            print("Error: Input file must contain three or more columns.")
            return

        data_subset = df.iloc[:, 1:]

        for col in data_subset.columns:
            if not np.issubdtype(data_subset[col].dtype, np.number):
                print(f"Error: Column '{col}' contains non-numeric values.")
                return

        weights = [float(w) for w in weights_str.split(',')]
        impacts = impacts_str.split(',')

        if len(weights) != data_subset.shape[1] or len(impacts) != data_subset.shape[1]:
            print("Error: Number of weights, impacts, and numeric columns must be the same.")
            return

        if not all(i in ['+', '-'] for i in impacts):
            print("Error: Impacts must be either '+' or '-'.")
            return

        matrix = data_subset.values.astype(float)
        norm_matrix = matrix / np.sqrt((matrix**2).sum(axis=0))
        weighted_matrix = norm_matrix * weights

        best = []
        worst = []
        for i in range(len(impacts)):
            if impacts[i] == '+':
                best.append(np.max(weighted_matrix[:, i]))
                worst.append(np.min(weighted_matrix[:, i]))
            else:
                best.append(np.min(weighted_matrix[:, i]))
                worst.append(np.max(weighted_matrix[:, i]))

        dist_best = np.sqrt(((weighted_matrix - best)**2).sum(axis=1))
        dist_worst = np.sqrt(((weighted_matrix - worst)**2).sum(axis=1))

        score = dist_worst / (dist_best + dist_worst)
        df['Topsis Score'] = score
        df['Rank'] = df['Topsis Score'].rank(ascending=False).astype(int)

        df.to_csv(output_file, index=False)
        print(f"Success: Results saved to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

Writing topsis.py


In [2]:
import pandas as pd
data = {
    'Model': ['M1', 'M2', 'M3', 'M4', 'M5'],
    'P1': [0.8, 0.7, 0.9, 0.6, 0.95],
    'P2': [15, 20, 10, 25, 12],
    'P3': [5, 4, 3, 2, 5],
    'P4': [40, 35, 50, 45, 60]
}
pd.DataFrame(data).to_csv('data.csv', index=False)

!python topsis.py data.csv "1,1,1,2" "+,-,+,-" result.csv

print("\n--- Result Table ---")
print(pd.read_csv('result.csv'))

Success: Results saved to result.csv

--- Result Table ---
  Model    P1  P2  P3  P4  Topsis Score  Rank
0    M1  0.80  15   5  40      0.762616     1
1    M2  0.70  20   4  35      0.634548     2
2    M3  0.90  10   3  50      0.566355     3
3    M4  0.60  25   2  45      0.329836     5
4    M5  0.95  12   5  60      0.517303     4


In [3]:
import os

PKG_NAME = "Topsis-FirstName-RollNumber"

os.makedirs(f"{PKG_NAME}/src/topsis_pkg", exist_ok=True)
!cp topsis.py {PKG_NAME}/src/topsis_pkg/topsis_logic.py

with open(f"{PKG_NAME}/src/topsis_pkg/__init__.py", "w") as f:
    f.write("from .topsis_logic import main")

with open(f"{PKG_NAME}/README.md", "w") as f:
    f.write("# TOPSIS Solver\nCommand line tool for MCDM.")

with open(f"{PKG_NAME}/pyproject.toml", "w") as f:
    f.write(f"""
[build-system]
requires = ["setuptools>=42"]
build-backend = "setuptools.build_meta"

[project]
name = "{PKG_NAME}"
version = "1.0.0"
dependencies = ["pandas", "numpy"]
""")

In [4]:
!pip install flask flask-mail subprocess.run

from flask import Flask, request
from flask_mail import Mail, Message
import subprocess

app = Flask(__name__)

app.config['MAIL_SERVER'] = 'smtp.gmail.com'
app.config['MAIL_PORT'] = 465
app.config['MAIL_USE_SSL'] = True
app.config['MAIL_USERNAME'] = 'your-email@gmail.com'
app.config['MAIL_PASSWORD'] = 'your-app-password'
mail = Mail(app)

@app.route('/calculate', methods=['POST'])
def calculate():
    email = request.form.get('email')
    weights = request.form.get('weights')
    impacts = request.form.get('impacts')
    f = request.files['file']

    f.save("input.csv")

    try:
        subprocess.run(["python", "topsis.py", "input.csv", weights, impacts, "output.csv"])

        msg = Message("TOPSIS Results", sender=app.config['MAIL_USERNAME'], recipients=[email])
        with app.open_resource("output.csv") as fp:
            msg.attach("output.csv", "text/csv", fp.read())
        mail.send(msg)

        return "Results sent to email."
    except Exception as e:
        return str(e)

  Preparing metadata (setup.py) ... done
  Created wheel for subprocess.run: filename=subprocess.run-0.0.8-py3-none-any.whl size=5345 sha256=ee49ee692a443616101aeae284b8dab2cabdbd2ffcd2a4993169eae8ce235085
  Stored in directory: /root/.cache/pip/wheels/4a/bb/1f/6aa8c10e4be30601bb1e1372f8be5952e0c92ece9dbcd02c77
Successfully built subprocess.run


In [11]:
import shutil
# Make sure this name matches the folder name in your sidebar exactly
shutil.make_archive('Topsis_Package_Submission', 'zip', 'Topsis-FirstName-RollNumber')

'/content/Topsis_Package_Submission.zip'